In [ ]:
!pip install git+https://github.com/openai/shap-e.git --no-cache-dir

  Cloning https://github.com/openai/shap-e.git to /tmp/pip-req-build-t1gdhax2
  Running command git clone --filter=blob:none --quiet https://github.com/openai/shap-e.git /tmp/pip-req-build-t1gdhax2
  Resolved https://github.com/openai/shap-e.git to commit 50131012ee11c9d2617f3886c10f000d3c7a3b43
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-6ctq51lu/clip_6afe6e99171a4efc96d437466884e334
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-6ctq51lu/clip_6afe6e99171a4efc96d437466884e334
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 313.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB

In [ ]:
import torch
from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import decode_latent_mesh, create_pan_cameras, decode_latent_images, gif_widget
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class generator():
    def __init__(self):
        self._device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f'using device:\t{device}')
        self._xm = load_model('transmitter', device=self._device)
        self._model = load_model('text300M', device=self._device)
        self._diffusion = diffusion_from_config(load_config('diffusion'))

    def generate(self, prompt):
        batch_size = 4
        guidance_scale = 10.0
        latents = sample_latents(
            batch_size=batch_size,
            model= self._model,
            diffusion=self._diffusion,
            guidance_scale=guidance_scale,
            model_kwargs=dict(texts=[prompt] * batch_size),
            progress=True,
            clip_denoised=True,
            use_fp16=True,
            use_karras=True,
            karras_steps=64,
            sigma_min=1e-3,
            sigma_max=160,
            s_churn=0,
        )
        render_mode = 'nerf' # you can change this to 'stf'
        size = 64 # this is the size of the renders; higher values take longer to render.

        cameras = create_pan_cameras(size, device)
        for i, latent in enumerate(latents):
          images = decode_latent_images(self._xm, latent, cameras, rendering_mode=render_mode)
          display(gif_widget(images))

        for i, latent in enumerate(latents):
            t = decode_latent_mesh(self._xm, latent).tri_mesh()
            with open(f'generated_{i}.obj', 'w') as f:
                t.write_obj(f)

In [ ]:
gen = generator()
gen.generate('A mini train')

using device:	cuda


  0%|          | 0/64 [00:00<?, ?it/s]

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAANXV1dXU1dTU1dTU1NPU09TT1NTT09PT1NPT09PT0tLT0tPS1N…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAOHoJeLnJODnJ+DnJd/nJeHmJODmI9/mJ9/mJd7mJt3lJtzhI9…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAALCwsK+wsK+vr6+vrq2tra2trKytra2srKysraysrKysq6usrK…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAANTU1NXT09TT1NPT1NPT09LT09PS09LS09LS0tLR0tLR0dHR0t…

In [ ]:
a

NameError: name 'a' is not defined